<a href="https://colab.research.google.com/github/Sakshibisen11/MovieReviews/blob/main/MovieReviews_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-09-12 20:19:23--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-09-12 20:19:23--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb62c0e4ba3730680e095ece99a.dl.dropboxusercontent.com/cd/0/inline/Bs1KYVxw1WHBSVGD9tykcPmaDkJL_D1-l4XNkxKLwi519dTlKnKNdIQPqY2v8sV0tph9-YxNOFyEGbGmxDxQNGGGDCQqmizKSvJRBcml6N_OsbxGeq2tRW6ENGWCLI7mRVJ5sTm5TMKPiR2wsYeC5cWRz8SdYgbA0uUIs6kCIS1MWA/file# [following]
--2022-09-12 20:19:24--  https://ucb62c0e4ba3730680e095ece99a.dl.dropboxusercontent.com/cd/0/inline/Bs1KYVxw1WHBSVGD9tykcPmaDkJL_D1-l4XNkxKLw

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,GRU,Embedding,Dropout


In [4]:
def prepareData(dir):
  data=text_dataset_from_directory(dir)
  return data.map(
      lambda text,label:(regex_replace(text,'<br/>',''),label),
  )

In [5]:
train_data=prepareData('movie-reviews-dataset/train')
test_data=prepareData('movie-reviews-dataset/test')
for text_batch,label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'The quote I used for my summary occurs about halfway through THE GOOD EARTH, as a captain of a Chinese revolutionary army (played by Philip Ahn) apologizes to a mob for not having time to shoot MORE of the looters among them, as his unit has just been called back to the front lines. Of course, the next looter about to be found out and shot is the main character of the film, the former kitchen slave girl O-Lan (for whose portrayal Luise Rainer, now 99-years-old, won her second consecutive best actress Oscar).<br /><br />The next scene finds O-Lan dutifully delivering her bag of looted jewels to her under-appreciative husband, farmer Wang Lung (Paul Muni), setting in motion that classic dichotomy of a man\'s upward financial mobility being the direct inverse of his moral decline.<br /><br />For a movie dealing with subject matter including slavery, false accusations, misogyny, starvation, home invasion,

In [6]:
model=Sequential()

In [7]:
model.add(Input(shape=(1,),dtype="string"))

In [8]:
#convert string input to vector representation
max_tokens=1000  #tokenize only 1000 words.Rest other words are considered as out of vocabulary
max_len=100 #determines maximum length of the vector
vectorize_layer=TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_len,
)

In [9]:
#extract data from train_data object and train the vectorizor
train_texts=train_data.map(lambda text,label:text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens+1,128))
model.add(GRU(64))
model.add(Dense(64,activation="relu"))
model.add(Dense(1,activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [12]:
model.fit(train_data,epochs=2)

Epoch 1/2
782/782 [==============================] - 62s 76ms/step - loss: 0.5817 - accuracy: 0.6755
Epoch 2/2
782/782 [==============================] - 59s 75ms/step - loss: 0.4319 - accuracy: 0.8003


In [13]:
model.evaluate(test_data)

782/782 [==============================] - 19s 24ms/step - loss: 0.4129 - accuracy: 0.8063


[0.41285645961761475, 0.8063200116157532]

In [14]:
text="I loved this movie"

In [15]:
model.predict([text]) #positive review

array([[0.8547847]], dtype=float32)